# Benchmark comparing MongoDB sync vs. async

In [33]:
import time
import asyncio
import contextlib
from pymongo import MongoClient
from motor.motor_asyncio import AsyncIOMotorClient

@contextlib.contextmanager
def timer(description):
    start_time = time.time()
    yield
    end_time = time.time()
    print('{} took {:5.4f} seconds'.format(description, end_time - start_time))
    
sync_client = MongoClient().aiotest.testcol
aio_client = AsyncIOMotorClient(max_pool_size=10).aiotest.testcol
loop = asyncio.get_event_loop()

In [8]:
N = 1000

In [10]:
_ = sync_client.insert({'abc': i} for i in range(N))

## Synchronous Requests

In [34]:
with timer('sequential sync read'):
    for i in range(N):
        sync_client.find_one({'abc': i})

sequential sync read took 0.3535 seconds


## Asynchronous Requests

In [12]:
with timer('sequential async read'):
    for i in range(N):
        loop.run_until_complete(aio_client.find_one({'abc': i}))

sequential async read took 0.5558857917785645 seconds


Using an async for-loop is not much faster:

In [13]:
async def f():
    async for doc in aio_client.find():
        pass
    
with timer('sequential async read with cursor'):
    loop.run_until_complete(f())

sequential async read with cursor took 0.028409957885742188 seconds


In [14]:
for chunk_size in range(1, 20):
    with timer('parallel async read with {} parallel requests'.format(chunk_size)):
        futures = [col.find_one({'abc': i})]
        for i in range(N // chunk_size):
            # interesting part:
            futures = [col.find_one({'abc': (i * chunk_size) + j})
                       for j in range(chunk_size)]
            loop.run_until_complete(asyncio.gather(*futures))

parallel async read with 1 parallel requests took 0.616163969039917 seconds
parallel async read with 2 parallel requests took 0.35162997245788574 seconds
parallel async read with 3 parallel requests took 0.2948617935180664 seconds
parallel async read with 4 parallel requests took 0.26516294479370117 seconds
parallel async read with 5 parallel requests took 0.24994611740112305 seconds
parallel async read with 6 parallel requests took 0.24745702743530273 seconds
parallel async read with 7 parallel requests took 0.23832011222839355 seconds
parallel async read with 8 parallel requests took 0.23779797554016113 seconds
parallel async read with 9 parallel requests took 0.23258399963378906 seconds
parallel async read with 10 parallel requests took 0.231536865234375 seconds
parallel async read with 11 parallel requests took 0.22452688217163086 seconds
parallel async read with 12 parallel requests took 0.2237100601196289 seconds
parallel async read with 13 parallel requests took 0.21818184852600

In [3]:
sync_client.drop()